<a href="https://colab.research.google.com/github/maleehahassan/HIDA_Into_to_DL/blob/main/01_networks_as_code_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview: Wine Classification with Neural Networks

This notebook demonstrates how to classify wine samples into three categories using a neural network and the classic sklearn wine dataset. The workflow includes:

1. **Data Loading & Exploration**: The wine dataset is loaded and its features are inspected.
2. **Preprocessing**: The target labels are one-hot encoded for multiclass classification. Features are normalized using standard scaling to improve model performance.
3. **Data Splitting**: The data is split into training and test sets, stratified by class to ensure balanced representation.
4. **Model Building**: A simple feedforward neural network is constructed using TensorFlow/Keras, with one hidden layer and a softmax output for multiclass classification.
5. **Training**: The model is trained for 18 epochs, and training loss is visualized to monitor learning progress.
6. **Prediction & Evaluation**: Predictions are made on the test set, and the model’s performance is evaluated using a confusion matrix and accuracy score.

This notebook helps learners understand the full workflow of preparing data, building and training a neural network for multiclass classification, and evaluating results using visual tools.

In [ ]:
# Import the wine dataset from sklearn
from sklearn.datasets import load_wine

# Load the dataset into a variable
dataset = load_wine()

In [ ]:
# Display the dataset dictionary to inspect its contents
dataset

In [ ]:
# Show the feature names in the dataset
dataset['feature_names']

In [ ]:
# Import pandas for data manipulation
import pandas as pd

X = dataset['data']

# Assign the feature data to X and one-hot encode the target labels
y = pd.get_dummies(dataset["target"])

In [ ]:
# Print the type and shape of the feature matrix X
print(type(X), X.shape)

In [ ]:
# Print the type and shape of the one-hot encoded target y
print(type(y), y.shape)

In [ ]:
# Import StandardScaler for feature normalization
from sklearn.preprocessing import StandardScaler

# Fit the scaler to X and transform X to have zero mean and unit variance
scaler = StandardScaler().fit(X)
X_ = scaler.transform(X)

In [ ]:
# Print the mean values per column after scaling
print("mean values per column:\n", X_.mean(axis=-2))

In [ ]:
# Print the standard deviation values per column after scaling
print("std  values per column:\n", X_.std(axis=-2))

In [ ]:
# Import train_test_split to split data into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_, y ,test_size=0.2,
                                                    random_state=0,
                                                    shuffle=True,
                                                    stratify=y)


In [ ]:
# Print the shapes of the train and test splits for features and targets
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
# Set random seeds for reproducibility
from numpy.random import seed
seed(1)
from tensorflow.random import set_seed
set_seed(2)
from tensorflow import keras



In [ ]:
# Define a function to create the neural network model
def create_model(X_shape):
    # Input layer
    inputs = keras.Input(shape=X_shape[1])

    # Hidden layer with 16 units and ReLU activation
    hidden_layer = keras.layers.Dense(16, activation="relu")(inputs)
    #hidden_layer = keras.layers.Dense(32, activation="relu")(hidden_layer)

    # Output layer with 3 units (for 3 classes) and softmax activation
    output_layer = keras.layers.Dense(3, activation="softmax")(hidden_layer)

    # Create the model object
    model = keras.Model(inputs=inputs, outputs=output_layer)
    return model

In [ ]:
# Create the model using the training data shape
model = create_model(X_train.shape)

In [ ]:
# Display the model architecture
model.summary()

In [ ]:
# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy())

In [ ]:
# Train the model on the training data for 18 epochs
history = model.fit(X_train, y_train, epochs=18)

In [ ]:
# Import seaborn for plotting
import seaborn as sns

# Plot the training loss over epochs
sns.lineplot(x=history.epoch, y=history.history['loss'])

In [ ]:
# Make predictions on the test data
y_pred = model.predict(X_test)

# Create a DataFrame for the predictions
prediction = pd.DataFrame(y_pred, columns=y.columns)

# Display the predictions
prediction

In [ ]:
# Get the predicted species by finding the class with the highest probability
predicted_species = prediction.idxmax(axis="columns")
predicted_species

In [ ]:
# Import confusion_matrix and accuracy_score for model evaluation
from sklearn.metrics import confusion_matrix, accuracy_score

# Get the true species from the test set
true_species = y_test.idxmax(axis="columns")

# Compute the confusion matrix
matrix = confusion_matrix(true_species, predicted_species)

# Display the confusion matrix
print(matrix)

In [ ]:
# Calculate and print the accuracy of the model
accuracy_score(true_species, predicted_species)